<a href="https://colab.research.google.com/github/MarianoChic09/Agentes-Inteligentes/blob/main/MSc_AI_ORT_Taller_IA_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller de Deep Learning - Tarea 1

Introducción:
El objetivo de esta tarea es evaluar su conocimiento sobre Deep Learning aplicado a un caso de uso real. En particular, vamos a evaluar la performance de sus modelos en una tarea de clasificación de escenas.

Dataset:
El dataset a ser utilizado consiste de más de 13.000 imágenes de entrenamiento agrupadas
en 6 posibles “escenarios”: edificios, bosques, glaciares, montañas, océanos, calles.
Pueden descargarlo en el siguiente link.
El mismo contiene dos carpetas (train_set y test_set) donde las imágenes están
agrupadas en sub carpetas de acuerdo a su categoría.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# ! cp "/content/drive/My Drive/data/scene_classification.zip" .
# ! unzip -q scene_classification.zip
# ! rm scene_classification.zip

cp: cannot stat '/content/drive/My Drive/data/scene_classification.zip': No such file or directory
unzip:  cannot find or open scene_classification.zip, scene_classification.zip.zip or scene_classification.zip.ZIP.
rm: cannot remove 'scene_classification.zip': No such file or directory


In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification"
%ls

/content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification
test_set/  train_set/


## Cargando el dataset

In [4]:
import os

print("Directorio actual:", os.getcwd())
print("\nContenido de 'train_set':", os.listdir("train_set"))

Directorio actual: /content/drive/MyDrive/Colab Notebooks/Datasets/scene_classification

Contenido de 'train_set': ['street', 'forest', 'mountain', 'buildings', 'glacier', 'sea']


In [5]:
# Reemplaza 'class_1' con el nombre de uno de tus subdirectorios.
print("Contenido de 'train_set/buildings':", os.listdir("train_set/buildings"))

Contenido de 'train_set/buildings': ['19513.jpg', '19447.jpg', '19456.jpg', '1959.jpg', '19460.jpg', '1954.jpg', '19584.jpg', '1964.jpg', '19638.jpg', '19608.jpg', '19455.jpg', '19526.jpg', '19416.jpg', '19556.jpg', '19623.jpg', '19446.jpg', '19407.jpg', '1943.jpg', '19466.jpg', '2059.jpg', '2099.jpg', '1979.jpg', '2170.jpg', '19901.jpg', '19778.jpg', '19969.jpg', '19830.jpg', '19923.jpg', '20048.jpg', '19913.jpg', '2083.jpg', '19895.jpg', '19649.jpg', '19848.jpg', '2076.jpg', '19976.jpg', '2136.jpg', '19728.jpg', '19806.jpg', '19963.jpg', '19996.jpg', '19855.jpg', '20027.jpg', '1977.jpg', '20054.jpg', '20011.jpg', '19713.jpg', '2144.jpg', '19802.jpg', '2002.jpg', '19915.jpg', '19701.jpg', '19896.jpg', '2038.jpg', '19674.jpg', '19689.jpg', '2108.jpg', '19973.jpg', '19661.jpg', '2171.jpg', '19776.jpg', '19846.jpg', '19910.jpg', '2158.jpg', '19816.jpg', '19874.jpg', '19789.jpg', '20017.jpg', '2114.jpg', '2088.jpg', '19666.jpg', '19722.jpg', '2069.jpg', '19800.jpg', '19735.jpg', '19978.jp

In [6]:
all_extensions = set()  # Para almacenar todas las extensiones encontradas

# Iterar sobre cada subdirectorio en "train_set"
for folder in os.listdir("train_set"):
    # Determinar las extensiones de archivos en el subdirectorio actual
    extensions = {os.path.splitext(file)[1].lower() for file in os.listdir(f"train_set/{folder}")}
    all_extensions.update(extensions)

    print(f"Para la carpeta {folder}:")
    print(extensions)
    print("-" * 50)

print("Todas las extensiones encontradas:", all_extensions)

Para la carpeta street:
{'.jpg'}
--------------------------------------------------
Para la carpeta forest:
{'.jpg'}
--------------------------------------------------
Para la carpeta mountain:
{'.jpg'}
--------------------------------------------------
Para la carpeta buildings:
{'.jpg'}
--------------------------------------------------
Para la carpeta glacier:
{'.jpg'}
--------------------------------------------------
Para la carpeta sea:
{'.jpg'}
--------------------------------------------------
Todas las extensiones encontradas: {'.jpg'}


In [7]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
print(os.listdir())
train_dataset = ImageFolder("train_set")#, extensions=['.jpg', '.jpeg', '.png'])
test_dataset = ImageFolder("test_set")

# Batch size de ejemplo.
BATCH_SIZE = 10

# Separamos en train y validation
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train, validation = torch.utils.data.random_split(train_dataset, [train_size,
valid_size])
print(f"{len(train)} Training images, {len(validation)} Validation images")

# Podemos usar data loaders como vimos en el práctico.
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True,
pin_memory=True)
val_loader = DataLoader(validation, batch_size=BATCH_SIZE, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True)

['train_set', 'test_set']
11227 Training images, 2807 Validation images


In [16]:
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from sklearn.metrics import accuracy_score, confusion_matrix
import time

class CustomCNN(nn.Module):
  def __init__(self, in_channels,number_classes):
    # in_channels: int, cantidad de canales de la imagen original
    super(CustomCNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels = 32, kernel_size = 3,padding=1)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3,padding=1)
    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3,padding=1)
    self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3,padding=1)
    self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 120, kernel_size = 3,padding=1)

    self.linear1 = nn.Linear(in_features = 120*4*4, out_features = 512)
    self.linear2 = nn.Linear(in_features = 512, out_features = number_classes)

    self.max_pooling = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = F.relu(self.conv2(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv3(out))
    out = F.relu(self.conv4(out))
    out = self.max_pooling(out)

    out = F.relu(self.conv5(out))
    out = self.max_pooling(out)

    out = self.dropout(out.flatten(1))

    out = F.relu(self.linear1(out))
    out = self.dropout(out)
    out = self.linear2(out)
    return out

In [17]:
# def get_dataloaders(train_transf, test_transf, batch_size):

#   train_dataset = CIFAR10("data", train=True, download=True, transform=train_transf)
#   test_dataset = CIFAR10("data", train=False, download=True, transform=test_transf)

#   train_size = int(0.8 * len(train_dataset))
#   valid_size = len(train_dataset) - train_size
#   train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

#   train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
#   valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
#   test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

#   return train_loader, valid_loader, test_loader

def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    for images, labels in loader:
      all_labels.extend(labels.numpy())

      optim.zero_grad()

      predictions = training_model(images.to(device))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(device))

      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
      for images, labels in loader:
        all_labels.extend(labels.numpy())

        predictions = val_model(images.to(device))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(device))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [18]:
# Global models config

BATCH_SIZE = 128
LR = 0.001
NUMBER_EPOCHS = 15
criterion = nn.CrossEntropyLoss().to(device)

In [20]:
# Fijamos las semillas siempre para poder comparar.

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los dataloaders
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = ImageFolder("train_set", transform=train_transform)
test_dataset = ImageFolder("test_set", transform=test_transform)

# train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

# Definimos el modelo y el optimizador
modelo_sin_aug = CustomCNN(3,10).to(device)
optimizer = torch.optim.Adam(modelo_sin_aug.parameters(), lr=LR)

# Entrenamos
train_model(modelo_sin_aug, train_loader, val_loader, criterion, optimizer, NUMBER_EPOCHS)

TypeError: ignored